In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_rows',999)
import numpy as np
import os
import warnings
#import unicodedata
warnings.filterwarnings("ignore")
#from io import BytesIO
#import gzip
import requests
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Percentage of firms that select 'Transportation' as the BIGGEST OBSTACLE

In [78]:
es = pd.read_excel('files/ES.xlsx' , sheet_name='Biggest Obstacle')

In [79]:
#Creo una lista con nombres de países de LATAM para filtrar la base es

cwpops = pd.read_excel('files/cwpops.xlsx', index_col = [0,1])
cwpops.reset_index(inplace=True)
n = list(cwpops['Country'].unique())

distances = pd.read_excel('files/distances.xlsx')
n = n + list(distances['Country'].unique())
n = list(set(n))

In [80]:
es = es.replace({'Brazil':'Brasil'})

In [81]:
es_latam = es.loc[es['Economy'].isin(n),]

In [83]:
es_latam['Year'] = es_latam['Year'].astype('Int64')

In [84]:
indices = list(es_latam[['Year' , 'Subgroup Level']].loc[(es_latam['Subgroup'] == 'Location')].reset_index().groupby('Subgroup Level').max('Year')['index'])

In [85]:
es_latam_un = es_latam.loc[indices,]

es_latam_un.rename(columns={"Economy": "Country", "Subgroup Level": "City"},inplace=True)

es_latam_un.set_index(['Country' , 'City'], inplace=True)

In [86]:
es_latam_un.drop(['Subgroup' , 'Top Subgroup Level' , 'Average/SE/N'], axis=1,inplace=True)

In [87]:
es_latam_un.drop(es_latam_un.columns[:-1], axis=1 , inplace=True)

## Velocidad de descarga de internet

In [92]:
mlab = pd.read_csv('files/mlab-cities.csv')

In [93]:
mlab = mlab.replace({'Brazil':'Brasil'})

In [94]:
mlab.set_index(['Country' , 'City'],inplace=True)

In [95]:
mlab.drop(mlab.columns[1:], axis=1 , inplace=True)

In [295]:
#n = list(es_latam['Economy'].unique())

#n[-1] = 'Venezuela'

In [96]:
a = mlab.reset_index()
mlab_lat = a.loc[a['Country'].isin(n),].set_index(['Country' , 'City'])

## Distancias: Accesibilidad y tiempo de viaje

Estas 2 variables surgen del Observatorio de Movilidad Urbana de CAF (OMU). Tiempo de viaje está disponible en CSV en este [link](https://www.caf.com/app_omu/#graphic?i0=superf_area_metropol_km2_urba&i5=num_vehic_micro_bus&i2=viajes_hab_dia_colectivo&i3=tiempo_hs_hab_dia). El indicador de accesibilidad habría que pedírselo a Guille. 

De cualquier manera, los dejamos fuera a ambos del análisis por ahora porque el indicador de tiempo de vaije posee pocas ciudades

In [3]:
tpv = pd.read_csv(r'files/omu - caf/tpv.csv' , encoding = 'ISO-8859-1')

In [4]:
tpv['city'] = tpv['city'].map(lambda x: str(x)[:-7])

In [5]:
tpv.rename(columns={'city': 'City' , 'Tiempos y distancias :: Tiempos promedio de viaje Omnibus (minutos / viajes)' : 'Tiempos promedio de viaje Omnibus (minutos / viajes)'} , inplace=True)
tpv.set_index('City',inplace=True)

In [10]:
tpv

,Tiempos promedio de viaje Omnibus (minutos / viajes)
City,
B. Horizonte,39.000
Bogotá,73.000
Brasilia,87.200
Buenos Aires,36.000
Caracas,67.000
Cd. de México,50.000
Curitiba,38.800
Florianópolis,29.600
Guadalajara,47.800


## Join de estas DF's

In [101]:
joined_df = es_latam_un.join(mlab_lat, how='outer').dropna()

In [102]:
joined_df.sort_index()

Percent of firms choosing transportation as their biggest obstacle  \
Country            City                                                                                
Argentina          Buenos Aires                                                  1.9                   
                   Mendoza                                                         0                   
                   Rosario                                                       3.8                   
Bolivia            Cochabamba                                                      0                   
                   La Paz                                                        8.2                   
                   Santa Cruz                                                    2.4                   
Brasil             Goias                                                           0                   
                   Mato Grosso                                                     0                   
                   Parana                                                          0                   
                   Rio de Janeiro                                               10.6                   
                   Sao Paulo                                                       0                   
Chile              Antofagasta                                                   0.7                   
                   Los Lagos                                                    17.2                   
                   Santiago                                                     10.4                   
Colombia           Barranquilla                                                  0.6                   
                   Bogota                                                        2.4                   
                   Cali                                                          0.1                   
                   Cartagena                                                     1.5                   
                   Medellin                                                      2.3                   
Costa Rica         San Jose                                                      2.6                   
Dominican Republic Santo Domingo                                                   2                   
Ecuador            Guayas                                                          0                   
                   Pichincha                                                     1.5                   
El Salvador        San Salvador                                                  0.6                   
Guatemala          Guatemala City                                                0.6                   
Honduras           San Pedro Sula                                                  0                   
                   Tegucigalpa                                                   5.6                   
Mexico             Guanajuato                                                    0.4                   
                   Puebla                                                          0                   
                   Veracruz                                                        0                   
Nicaragua          Managua                                                       0.3                   
Panama             Colon                                                         4.2                   
                   Panama City                                                   0.2                   
Paraguay           Central                                                       7.4                   
Peru               Arequipa                                                      2.5                   
                   Chiclayo                                                      3.7                   
                   Lima                                                          3

In [103]:
joined_df.to_csv('Entregables/metricas/conect_int.csv')